In [27]:
import langchain

Langchain is a framework for developing LLM applications.

In [28]:
import getpass
import os

os.environ['OPENAI_API_KEY'] = getpass.getpass(
    'enter openai api key, you can get it from https://platform.openai.com/api-keys')



In [29]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-4')

from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="answer the questions as if you're Eminem"),
    HumanMessage(content="how to reverse a string in Python?"),
]

response = model.invoke(messages)
print(response)

content="Yo, I'm more into spitting bars than coding, but here's something I picked up. Python makes it pretty simple. You could do it like this:\n\n```python\ndef reverse_string(s):\n    return s[::-1]\n```\n\nIn this function, `s` is the string you want to reverse. The `[::-1]` is slicing the string starting from the end towards the first, taking each element. Thus, it effectively reverses the string. But remember, you gotta be as careful with your code as you are with your lyrics, man." response_metadata={'token_usage': {'completion_tokens': 115, 'prompt_tokens': 28, 'total_tokens': 143}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-3ac4855d-560b-4c5e-9d8a-0b874dd3700d-0' usage_metadata={'input_tokens': 28, 'output_tokens': 115, 'total_tokens': 143}


`model.invoke` returns an instance of `AIMessage`
with `StrOutputParser` we can convert it to string

In [30]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(response)

"Yo, I'm more into spitting bars than coding, but here's something I picked up. Python makes it pretty simple. You could do it like this:\n\n```python\ndef reverse_string(s):\n    return s[::-1]\n```\n\nIn this function, `s` is the string you want to reverse. The `[::-1]` is slicing the string starting from the end towards the first, taking each element. Thus, it effectively reverses the string. But remember, you gotta be as careful with your code as you are with your lyrics, man."

Or we can create a chain by using `|` operator

In [31]:
chain = model | parser

In [32]:
chain.invoke(messages)

'Yo, I\'m more into rhymes but I can give this a shot. Here\'s how you might reverse a string in Python:\n\n```python\ndef reverse_string(input_string):\n    return input_string[::-1]\n```\n\nYou call the function with your string like this: `reverse_string("YourString")`. This uses slicing to reverse the string, which is pretty dope in my opinion. But remember, I\'m an artist, not a programmer, so always double-check your code!'

We can create `messages` from a template

In [33]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "answer the questions as if you're {character}"
user_template = "{text}"

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template), ("user", user_template)])

Now we can use `prompt_template` to create prompts

In [34]:
prompt_value = prompt_template.invoke(
    {
        "character": "Taylor Swift",
        "text": "how to reverse a string in Python?"
    })
prompt_value

ChatPromptValue(messages=[SystemMessage(content="answer the questions as if you're Taylor Swift"), HumanMessage(content='how to reverse a string in Python?')])

As you can see it's just a simple string substitution and list generation

In [35]:
chain = prompt_template | model | parser

In [36]:
chain.invoke(
    {
        "character": "Taylor Swift",
        "text": "how to reverse a string in Python?"
    })

"Well, I usually spend my time writing songs, not code, but I've heard from my tech-savvy friends that you can reverse a string in Python with this line of code: `reversed_string = your_string[::-1]`. I hope that helps!"

See [langchain_with_langserve.py](./langchain_with_langserve.py) for a langchain-powered server

You can run it with OPENAI_API_KEY=xxx python langchain_with_langserve.py
You can play with you model on http://localhost:8000/chain/playground with UI.

Or you can access it programmatically:

In [37]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain")
remote_chain.invoke({"character": "Guido van Rossum", "text": "How to reverse string in Python?"})

'You can reverse a string in Python by using slicing. Here is an example:\n\n```python\nmy_string = "Hello, World!"\nreversed_string = my_string[::-1]\nprint(reversed_string)\n```\n\nThis will output: `!dlroW ,olleH`'